In [ ]:

manualMasks = []
dataFileNames = glob.glob("./manualSegmentation" + "/*" + ImageExtension) #Obtain filenames for each set
for j in range(0,numberTestFiles):  
    name = re.sub(testingDataImagesPath, '', dataFileNames[0])
    name = re.sub('originalSegmentation','manualSegmentation',name)
    manualMasks.append(Image.open(name).convert('L'))
    

    
    #Extract the information needed to grab the manual version


mseMasks = []
ssimMasks = []
for i in range(0, len(StopPercentageTestingSLADSArr)):
    mseMask = []
    ssimMask = []
    for j in range(0,numberTestFiles):
        testImageFileName = dataFileNames[j]       
        reconstructionMask = np.asarray(all_reconstructionMasks[i][j])[0]
        reconstructionMask = Image.fromarray(test*255)
        manualMask = manualMasks[j]
        
        mseMaskValue = #Calculate MSE
        ssimMaskValue = #Calculate SSIM
        mseMask.append(mseMaskValue)
        ssimMask.append(ssimMaskValue)
    mseMasks.append(mseMask)
    ssimMasks.append(ssimMask)

#Make plot of MSE, each percentage as series



#Make plot of SSIM, each percentage as series    
    
    



In [ ]:
#FILTER ERDVALUESNP BY RECONIMAGE CONTOUR RESULTS; FINDNEWMEASUREMENTIDXS FUNCTION
#NOTE THAT ERDVALUESNP IS DERIVED FROM THE RECONIMAGE, SO UNLESS SOMETHING HAS GONE WRONG, THIS SHOULDN'T BE NECCESSARY

            
            #Find the largest area in recon image; filter ERDValuesNP with it (nan for absent)
            #filterArray = np.empty([SizeImage[0],SizeImage[1]])
            #filterArray[:] = np.nan
            #contours, hierarchy = cv2.findContours(ReconImage.astype('uint8'), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE) #Identify areas
            #areas = [cv2.contourArea(c) for c in contours] #Save the areas into an object
            #filterArray = cv2.drawContours(filterArray, [contours[np.argmax(areas)]], -1, (255, 255, 255), -1) > 0 #Draw largest area
            #filteridxs = np.argwhere(filterArray).tolist()
            #xValues = tuple([x[0] for x in filteridxs])
            #yValues = tuple([y[1] for y in filteridxs])
            #filteredERDValuesNP = np.empty([SizeImage[0],SizeImage[1]])
            #filteredERDValuesNP[:] = np.nan
            #filteredERDValuesNP[xValues, yValues] = ERDValuesNP[xValues, yValues]
            
            #Choose the line with most information gain, both with and without filtering by area
            #lineERDValues = []
            #lineERDValuesFiltered = []
            #for line in InitialMaskObject.linesToScan:
            #    xValues = tuple([x[0] for x in line])
            #    yValues = tuple([y[1] for y in line])
            #    if InitialMaskObject.useERDmean:
            #        lineERDValues.append(np.nanmean(ERDValuesNP[xValues, yValues]))
            #        lineERDValuesFiltered.append(np.nanmean(filteredERDValuesNP[xValues, yValues]))
            #    else:
            #        lineERDValues.append(np.nansum(ERDValuesNP[xValues, yValues]))
            #        lineERDValuesFiltered.append(np.nansum(filteredERDValuesNP[xValues, yValues]))

            #if InitialMaskObject.partialLine:
            #    idxs = InitialMaskObject.linesToScan[np.nanargmax(lineERDValuesFiltered)]
            #    for idx in idxs:
            #        if not np.isnan(filteredERDValuesNP[tuple(idx)]): 
            #            NewIdxs.append(idx)
            #else:
            #    NewIdxs = InitialMaskObject.linesToScan[np.nanargmax(lineERDValues)]

            #If this procedure yields no new pixels to scan fall back on full line scanning
            #if len(NewIdxs) <= 0:
            #    NewIdxs = InitialMaskObject.linesToScan[np.nanargmax(lineERDValues)]
            
            
            
#PARALLIZE NEW IDX FINDING WITHIN THE FINDNEWMEASUREMENTIDXS FUNCTION
#THIS WAS FOUND TO SIGNFICANTLY DECREASE PROGRAM EFFICIENCY (17 SECONDS) AND WAS REMOVED

def idxFinder_parhelper(InitialMaskObject, ERDValuesNP, testResult, lineToScan, lineNum):
    line = InitialMaskObject.linesToScan[lineNum]
    ERDPtsToScan = []
    sumLineERDValue = np.nan
    ptsList = []
    for pt in line:
        if pt in lineToScan:
            ERDPtsToScan.append(pt)
        if testResult[tuple(pt)]:
            if np.isnan(sumLineERDValue):
                sumLineERDValue = ERDValuesNP[tuple(pt)]
            else:
                sumLineERDValue = sumLineERDValue + ERDValuesNP[tuple(pt)]
            ptsList.append(pt)
    return ERDPtsToScan, sumLineERDValue, ptsList

#OLD TESTING ROUTINE, REPEATED CALCULATION EFFFORT, NO ANIMATION INTEGRATION, USED ARRAY OF POSSIBLE STOPPING PERCENTAGES
#OLD TESTING ROUTINE
    
    #For each of the possible training sampling percentages (each with their own best c)
    for p in tqdm(range(0,len(StopPercentageSLADSArr)), desc = 'Training Sampling Percentages', leave = True):
        StopPercentageSLADS = float(StopPercentageSLADSArr[p])
        
        #Import the best c and theta values; Training for database must have been performed first
        if not overrideBestC: #If automatic best c selection
            LoadPath_bestc = trainingFeaturesPath + 'TrainingDB_' + str(TestingImageSet) + os.path.sep + 'best_c' + '_StopPerc_' + str(StopPercentageSLADS) + '.npy'
            if not os.path.exists(LoadPath_bestc):
                sys.exit('Error!!! The best c file ' + SavePath_bestc + ' does not exist. Check entry for ' + SavePath_bestc)
            c = np.load(LoadPath_bestc)[0].astype(float)
        else: #If manual best c selection
            c = overrideBestCValue.astype(float)

        #Directory setup for Theta
        ThetaLoadPath = trainingFeaturesPath + TrainingDBName + os.path.sep + 'c_' + str(c) + os.path.sep
        if not os.path.exists(ThetaLoadPath):                                                                                                                          
            sys.exit('Error!!! Check folder ./ResultsAndData/TrainingSavedFeatures/TrainingDB_' + TrainingImageSet + ' for folder c_' + str(c))
        
        #Load Theta
        Theta=np.transpose(np.load(ThetaLoadPath +'Theta' + '_StopPerc_' + str(StopPercentageSLADS) + '.npy'))

        Beta = computeBeta(SizeImage)
        StopCondParamsObject=StopCondParams()
        
        #Create an object to hold progressive development of model
        testingResultsAverageObject = simulationResults() 
        testingResultsAverageObject.initialize()
        
        #Create an object to hold all of the testing results for each sampling percentage
        testingResults = []
        
        all_MSE_testingResults = []
        all_SSIM_testingResults = []
        all_Distort_testingResults = []
        all_reconstructionMasks = []
        
        #For each of the possible testing sampling percentages
        for q in tqdm(range(0,len(StopPercentageTestingSLADSArr)), desc = 'Testing Sampling Percentages', leave = True):
            StopPercentageTestingSLADS = float(StopPercentageTestingSLADSArr[q])
            StopCondParamsObject.initialize(Beta,StoppingThrehsold,50,2,StopPercentageTestingSLADS)
            dataFileNames = glob.glob(testingDataImagesPath + "/*" + ImageExtension) #Obtain filenames for each set
            numberTestFiles = len(testingData)

            #Perform SLADS on all of the images, saving statistics of interest in parallel
            testingResultObject = Parallel(n_jobs=num_threads)(delayed(stats_parhelper)(num_threads, InitialMaskObject, loadOrGenerateInitialMask(loadPathInitialMask,MaskType,InitialMaskObject,SizeImage,lineScanning), codePath, TestingImageSet, SizeImage, StopCondParamsObject, Theta, TrainingInfoObject, TestingInfoObject, Resolution, ImageType, UpdateERDParamsObject, BatchSamplingParamsObject, testingOutputName(testingFeaturesPath, dataFileNames[i], StopPercentageSLADS, StopPercentageTestingSLADS), i, ImageExtension, PlotResult, Classify, errorPlot, isRunningParallel, lineScanning, simulation, testingData[i]) for i in tqdm(range(0,numberTestFiles), desc = 'Testing Images', leave = True))
            
            #Create holding arrays for the results
            MSE_testingResults = []
            SSIM_testingResults = []
            Distort_testingResults = []
            reconstructionMasks = []
            
            #Extract results from returned object
            for result in testingResultObject: 
                MSE_testingResults.append(result.mseError)
                SSIM_testingResults.append(result.ssimError)
                Distort_testingResults.append(result.totalDistortion)
                reconstructionMasks.append(result.masks)
                
            #Store the Average DMs for all images for a particular testing sampling percentage
            testingResultsAverageObject.saveAverageErrorData(np.mean(MSE_testingResults), np.mean(SSIM_testingResults), np.mean(Distort_testingResults))
            
            all_MSE_testingResults.append(MSE_testingResults)
            all_SSIM_testingResults.append(SSIM_testingResults)
            all_Distort_testingResults.append(Distort_testingResults)
            all_reconstructionMasks.append(reconstructionMasks)
            
            #Directory setup for individual images final save path
            ImagesFinalSavePath = ImagesSavePath + 'StopTrainPerc_' + str(StopPercentageSLADS) + '_StopTestPerc_' + str(StopPercentageTestingSLADS) + os.path.sep
            if os.path.exists(ImagesFinalSavePath): 
                shutil.rmtree(ImagesFinalSavePath)
            os.makedirs(ImagesFinalSavePath)

            #Obtain filenames for each of the images that were made
            dataFileNames = glob.glob(testingFeaturesPath + "/*"+ImageExtension) 
            for j in range(0, len(dataFileNames)): #For each of the files
                outputName = dataFileNames[j] #Grab the filename
                outputName = re.sub(testingFeaturesPath, '', outputName) #Remove the directory prefix
                shutil.move(dataFileNames[j], ImagesFinalSavePath+outputName) #Move them to an Images folder

        #Plot the average DMs across the different testing sampling percentages given a particular training sampling percentage
        testingSpecificStatisticsSavePathSuffix ='_StopTrainPerc_' + str(StopPercentageSLADS)
        testingSpecificStatisticsSavePath = testingStatisticsSavePath + 'StopTrainPerc_' + str(StopPercentageSLADS) + os.path.sep    
        
        #Directory setup for specific testing statistics
        if os.path.exists(testingSpecificStatisticsSavePath): 
            shutil.rmtree(testingSpecificStatisticsSavePath)
        os.makedirs(testingSpecificStatisticsSavePath)      
        
        plotErrorData(testingSpecificStatisticsSavePath, testingSpecificStatisticsSavePathSuffix, testingResultsAverageObject, StopPercentageTestingSLADSArr.tolist(), testingPlotTitle, plotXLabel) #Plot and save the error data obtained during 
        trainTestAverageErrors.append(testingResultsAverageObject)
        
    #Plot the average DMS together in a single plot for all of the training sampling percentages
    ttPlotAverageErrors(testingStatisticsSavePath, StopPercentageSLADSArr, StopPercentageTestingSLADSArr, trainTestAverageErrors)

    #Plot MSE and SSIM distribution
    names = []
    for i in range(0, len(StopPercentageTestingSLADSArr)):
        names.append(str(StopPercentageTestingSLADSArr[i]) + '%')

    f = plt.figure(figsize=(20,8))
    plt.hist(all_MSE_testingResults, bins=10, align='left', label=names)
    plt.legend()
    plt.legend(bbox_to_anchor=(1, 1), loc='left', ncol=1)
    plt.ylabel('Frequency')
    plt.xlabel('MSE')
    plt.savefig(testingStatisticsSavePath + 'MSE_Distribution' + '.png')
    
    f = plt.figure(figsize=(20,8))
    plt.hist(all_SSIM_testingResults, bins=10, align='left', label=names)
    plt.legend()
    plt.legend(bbox_to_anchor=(1, 1), loc='left', ncol=1)
    plt.ylabel('Frequency')
    plt.xlabel('SSIM')
    plt.savefig(testingStatisticsSavePath + 'SSIM_Distribution' + '.png')
    
    
    
#AFTER INTENDED PROCEDURES (TRAINING/TESTING) HAVE BEEN PERFORMED
print('\n\n\n' + ('#' * int(consoleColumns)))
print('PROGRAM COMPLETE')
print('#' * int(consoleColumns) + '\n')





#OLD ANIMATION FUNCTION, REPEATED TESTING AT INDIVIDUAL PERCENTAGES




print('\n\n' + ('#' * int(consoleColumns)))
print('ANIMATING MODEL')
print(('#' * int(consoleColumns)) + '\n')

#Set initial percentages and adjustment for each step
if not lineScanning:
    StopPercentageTestingSLADS = 1
    percentageAdjustment = StopPercentageTestingSLADS
else:
    StopPercentageTestingSLADS = (SizeImage[0]/(SizeImage[0]*SizeImage[1]))*100
    percentageAdjustment = StopPercentageTestingSLADS

    
#Directory setup for animation images
AnimationImagesSavePath = ImagesSavePath + 'Animation' + os.path.sep
if os.path.exists(AnimationImagesSavePath): 
    shutil.rmtree(AnimationImagesSavePath)
os.makedirs(AnimationImagesSavePath)

#Perform testing for the images up to 50% of the total pixel count for generating an animation
while(StopPercentageTestingSLADS <= 50):
    StopPercentageTestingSLADS = float(StopPercentageTestingSLADS)
    StopCondParamsObject.initialize(Beta,StoppingThrehsold,50,2,StopPercentageTestingSLADS-percentageAdjustment)
 
    dataFileNames = glob.glob(testingDataImagesPath + "/*" + ImageExtension) #Obtain filenames for each testing set
    numberTestFiles = len(testingData)

    testingResultObject = []
    for i in tqdm(range(0,numberTestFiles), desc = 'Testing Images', leave = True):
        result = stats_parhelper(num_threads, InitialMaskObject, loadOrGenerateInitialMask(loadPathInitialMask,MaskType,InitialMaskObject,SizeImage,lineScanning), codePath, TestingImageSet, SizeImage, StopCondParamsObject, Theta, TrainingInfoObject, TestingInfoObject, Resolution, ImageType, UpdateERDParamsObject, BatchSamplingParamsObject, testingOutputName(testingFeaturesPath, dataFileNames[i], StopPercentageSLADS, StopPercentageTestingSLADS), i, ImageExtension, PlotResult, Classify, errorPlot, isRunningParallel, lineScanning, simulation, testingData[i])
        testingResultObject.append(result)

    #Create holding arrays for the results
    mseTestingResults = []
    ssimTestingResults = []
    distortTestingResults = []

    #Extract results from returned object
    for result in testingResultObject: 
        mseTestingResults.append(result.mseError)
        ssimTestingResults.append(result.ssimError)
        distortTestingResults.append(result.totalDistortion)

    #Obtain filenames for each of the images that were made
    dataFileNames = glob.glob(testingFeaturesPath + "/*"+ImageExtension) 
    for j in range(0, len(dataFileNames)): #For each of the files
        outputName = dataFileNames[j] #Grab the filename
        outputName = re.sub(testingFeaturesPath, '', outputName) #Remove the directory prefix
        shutil.move(dataFileNames[j], AnimationImagesSavePath+outputName) #Move them to the Animation images folder
    if not lineScanning:
        StopPercentageTestingSLADS += 1
    else: #lineScanning.
        StopPercentageTestingSLADS += percentageAdjustment

        
        
        
        





In [ ]:
#ANIMATION STUFF BELOW

In [ ]:
#Using resizeMe program, convert to jpegs into the Animation folder, then run sorting again
#For each of the testing images create a new subfolder and move their testing results into it
dataFileNames = glob.glob(testingDataImagesPath + "/*" + ImageExtension) #Obtain filenames for each set
for i in range(0, len(dataFileNames)):
    outputName = re.sub(testingDataImagesPath, '', dataFileNames[i])
    outputName = re.sub('.png', '', outputName)
    print(outputName)
    os.mkdir(AnimationImagesSavePath + outputName)
    animationImageFileNames = glob.glob(AnimationImagesSavePath + "/*" + '.jpg') 
    for j in range(0, len(animationImageFileNames)):
        if(outputName in animationImageFileNames[j]):
            shutil.move(animationImageFileNames[j], AnimationImagesSavePath + outputName)

        

In [ ]:

StopPercentageTestingSLADS = 0

#Directory setup for animation images
AnimationImagesSavePath = ImagesSavePath + 'Animation' + os.path.sep
if os.path.exists(AnimationImagesSavePath): 
    shutil.rmtree(AnimationImagesSavePath)
os.makedirs(AnimationImagesSavePath)

#Perform testing for the images up to 50% of the total pixel count for generating an animation
while(StopPercentageTestingSLADS <= 50):
    StopPercentageTestingSLADS = float(StopPercentageTestingSLADS)
    StopCondParamsObject.initialize(Beta,StoppingThrehsold,50,2,StopPercentageTestingSLADS)

    dataFileNames = glob.glob(testingDataImagesPath + "/*" + ImageExtension) #Obtain filenames for each testing set
    numberTestFiles = len(dataFileNames)

    #Perform SLADS on all of the images, saving statistics of interest in parallel
    testingResultObject = Parallel(n_jobs=num_threads)(delayed(stats_parhelper)(numberTestFiles, loadOrGenerateInitialMask(loadPathInitialMask,MaskType,InitialMaskObject,SizeImage,lineScanning), codePath, TestingImageSet, SizeImage, StopCondParamsObject, Theta, TrainingInfoObject, TestingInfoObject, Resolution, ImageType, UpdateERDParamsObject, BatchSamplingParamsObject, testingOutputName(testingFeaturesPath, dataFileNames[i], StopPercentageSLADS, StopPercentageTestingSLADS),SimulationRun, i, ImageExtension, PlotResult, Classify, dataFileNames[i], errorPlot, isRunningParallel) for i in tqdm(range(0,numberTestFiles), desc = 'Testing Images', leave = True))

    #Create holding arrays for the results
    mseTestingResults = []
    ssimTestingResults = []
    distortTestingResults = []

    #Extract results from returned object
    for result in testingResultObject: 
        mseTestingResults.append(result.mseError)
        ssimTestingResults.append(result.ssimError)
        distortTestingResults.append(result.totalDistortion)

    #Obtain filenames for each of the images that were made
    dataFileNames = glob.glob(testingFeaturesPath + "/*"+ImageExtension) 
    for j in range(0, len(dataFileNames)): #For each of the files
        outputName = dataFileNames[j] #Grab the filename
        outputName = re.sub(testingFeaturesPath, '', outputName) #Remove the directory prefix
        shutil.move(dataFileNames[j], AnimationImagesSavePath+outputName) #Move them to the Animation images folder

    StopPercentageTestingSLADS += 1 #Increase the stopping percentage


In [ ]:
#Manual segmentation comparison for masking sandstone

In [ ]:


originalFileNames = glob.glob(testingDataImagesPath + "/*" + ImageExtension)
testingDataPath = r'./manualSegmentation'
manualMaskNames = glob.glob(testingDataPath + "/*" + '.png') #Obtain filenames for each set
manualMasks = []

#For each of the testing images
for j in range(0,numberTestFiles-1):
    SLADS_Mask = all_reconstructionMasks[0][j]
    filename = originalFileNames[j]
    filename = re.sub('./ResultsAndData/TestingSavedFeatures/TestingDB_1/Images/', '', filename)     
    filename = re.sub('originalSegmentation','', filename)
    for k in range(0,len(manualMaskNames)-1):
        if (filename in manualMaskNames[k]):
            manual_Mask = manualMaskNames[k]
            manualMasks.append(manual_Mask)
    

all_MSE_results = []
all_SSIM_results = []

#For each of the percentages
for i in range (0, len(all_reconstructionMasks)-1):
    MSE_results = []
    SSIM_results = []
    
    #For each of the testing images
    for j in range(0,numberTestFiles-1):
        recMask = Image.fromarray(np.asarray(all_reconstructionMasks[i][j]).reshape(SizeImage)*255)
        recMask = PIL.ImageOps.invert(recMask)
        manMask = Image.open(manualMasks[j]).convert('L')
        MSE = np.mean((np.asarray(recMask).astype("float") - np.asarray(manMask).astype("float")) ** 2)
        SSIM = compare_ssim(np.asarray(recMask).astype("float"), np.asarray(manMask).astype("float"))
        MSE_results.append(MSE)
        SSIM_results.append(SSIM)
    all_MSE_results.append(MSE_results)
    all_SSIM_results.append(SSIM_results)



In [ ]:
#Masking MSE

names = []
for i in range(0, len(StopPercentageTestingSLADSArr)):
    names.append(str(StopPercentageTestingSLADSArr[i]) + '%')

f = plt.figure(figsize=(20,8))
plt.hist(all_MSE_testingResults, bins=10, align='left', label=names)
plt.legend()
plt.legend(bbox_to_anchor=(1, 1), loc='left', ncol=1)
plt.ylabel('Frequency')
plt.xlabel('MSE')


In [ ]:
#Masking SSIM
f = plt.figure(figsize=(20,8))
plt.hist(all_SSIM_testingResults, bins=10, align='left', label=names)
plt.legend()
plt.legend(bbox_to_anchor=(1, 1), loc='left', ncol=1)
plt.ylabel('Frequency')
plt.xlabel('SSIM')